In [2]:
import sys 
sys.path.append("..")

In [3]:
import numpy as np 
import pandas as pd
from hdna import *
from juliacall import Main as jl 

In [4]:
# Import experimental data from Hertel 
expdata = pd.read_csv('../data/herteldata.csv', names=['seq', 'expvalue'])
# Clean the dataframe 
expdata = expdata.drop(0)
expdata['expvalue'] = ['{:e}'.format(float(e)) for e in expdata['expvalue']]


In [5]:
rates = []
model = Model('dna', '3D')

In [6]:
for i, (seq, exp) in enumerate(zip(expdata['seq'], expdata['expvalue'])):
    seq = str(seq.strip())      # make sure they are alright
    exp = float(exp.strip())    #
    print(f'Creating network from string {seq}...')
    A = Strand(model, seq)
    B = A.complementary()
    kinet = Kinetwork(model, A, B, 1)
    geo = Geometry(120, 360)
    K = Kinetics(model, kinet, geo)
    opts = Options(method='direct', runtime=3e-6, Nmonte=20)
    print('embedding strand into biosimulator network model...')
    simulatore = Simulator(model, kinet, K, options=opts)
    print('start running simulations...')
    results = simulatore.ensemble()
    mfpt = simulatore.mfpts(results)
    rates.append(1/mfpt)
    print(f"experimental rate: {'{:e}'.format(exp)}")
    print(f"computed rate:     {'{:e}'.format(1/mfpt)}", '\n')
    if i == 0: break

Creating network from string GTTCGGTCTA...
embedding strand into biosimulator network model...
start running simulations...


100%|██████████| 20/20 [00:00<00:00, 23.27it/s]

3 simulations didn't produce a duplex.
That's 15.0% of simulations
experimental rate: 1.145950e+06
computed rate:     1.075396e+06 



In [7]:
react = pd.DataFrame()


In [8]:
simulatore.lt(str(list(simulatore.biosim.reaction_list[e].reactants)[0]))

NameError: name 'e' is not defined

In [9]:
for e in simulatore.biosim.reaction_list:
    print(e,'{:e}'.format(simulatore.biosim.reaction_list[e].rate), simulatore.lt(str(list(simulatore.biosim.reaction_list[e].reactants)[0])),simulatore.lt(str(list(simulatore.biosim.reaction_list[e].products)[0])))

f_nucleation1 3.850521e+07 ..........+.......... ..(.......+.......)..
b_nucleation1 1.439529e+07 ..(.......+.......).. ..........+..........
f_nucleation2 3.850521e+07 ..........+.......... .(........+........).
b_nucleation2 3.165815e+07 .(........+........). ..........+..........
f_nucleation3 3.850521e+07 ..........+.......... ......(...+...)......
b_nucleation3 3.018623e+07 ......(...+...)...... ..........+..........
f_nucleation4 3.850521e+07 ..........+.......... (.........+.........)
b_nucleation4 1.428910e+07 (.........+.........) ..........+..........
f_nucleation5 3.850521e+07 ..........+.......... ....(.....+.....)....
b_nucleation5 3.622386e+06 ....(.....+.....).... ..........+..........
f_nucleation6 3.850521e+07 ..........+.......... .......(..+..).......
b_nucleation6 8.279643e+05 .......(..+..)....... ..........+..........
f_nucleation7 3.850521e+07 ..........+.......... .........(+).........
b_nucleation7 3.014395e+07 .........(+)......... ..........+..........
f_nucl

In [10]:
import networkx as nx

In [11]:
R = simulatore.biosim.reaction_list
names = []
rates = []
reags = []
prods = []
for r in R:
    names.append(str(r))
    rates.append(R[r].rate)
    reags.append(simulatore.lt(str(list((R[r].reactants))[0])))
    prods.append(simulatore.lt(str(list((R[r].products))[0])))
RDF = pd.DataFrame([names, rates, reags, prods], index=['name', 'rate', 'reactants', 'products']).T

G = nx.from_pandas_edgelist(RDF, source='reactants', target='products', edge_attr=['name', 'rate'], create_using=nx.DiGraph())

In [12]:
a = [1,2,3,4,5,6,7]
for i, j in enumerate(a[:-1], start=1):
    print(j, a[i])


1 2
2 3
3 4
4 5
5 6
6 7


In [13]:
print(G.edges['..........+..........','.....(....+....).....'])
print(G.edges['.....(....+....).....','..........+..........'])

{'name': 'f_nucleation8', 'rate': 38505212.66779661}
{'name': 'b_nucleation8', 'rate': 1179884.085772729}


In [ ]:
for e in simulatore.biosim.reaction_list:
    print(e,'{:e}'.format(simulatore.biosim.reaction_list[e].rate), simulatore.lt(str(list(simulatore.biosim.reaction_list[e].reactants)[0])),simulatore.lt(str(list(simulatore.biosim.reaction_list[e].products)[0])))

f_nucleation1 3.850521e+07 ..........+.......... .....(....+....).....
b_nucleation1 1.179884e+06 .....(....+....)..... ..........+..........
f_nucleation2 3.850521e+07 ..........+.......... ...(......+......)...
b_nucleation2 2.027801e+06 ...(......+......)... ..........+..........
f_nucleation3 3.850521e+07 ..........+.......... ........(.+.)........
b_nucleation3 5.655053e+06 ........(.+.)........ ..........+..........
f_nucleation4 3.850521e+07 ..........+.......... .......(..+..).......
b_nucleation4 8.279643e+05 .......(..+..)....... ..........+..........
f_nucleation5 3.850521e+07 ..........+.......... ....(.....+.....)....
b_nucleation5 3.622386e+06 ....(.....+.....).... ..........+..........
f_nucleation6 3.850521e+07 ..........+.......... (.........+.........)
b_nucleation6 1.428910e+07 (.........+.........) ..........+..........
f_nucleation7 3.850521e+07 ..........+.......... ......(...+...)......
b_nucleation7 3.018623e+07 ......(...+...)...... ..........+..........
f_nucl

In [ ]:
print(simulatore.biosim.reaction_list["b_nucleation1"].origex)

ooooobooooAoooodooooo --> ooooooooooAoooooooooo + ooooooooooAoooooooooo


In [ ]:
simulatore.get_trajectory(results[1])

['..........+..........',
 '((........+...)).....',
 '(.((......+..)).)....',
 '((........+...)).....',
 '..(.......+.)........',
 '.(........+.)........',
 '..(.......+.)........',
 '((........+...)).....',
 '..(.......+.)........',
 '.(........+.)........',
 '..(.......+.)........',
 '((........+...)).....',
 '..(.......+.)........',
 '..........+..........',
 '........(.+...)......',
 '....(.((..+......)).)',
 '.....((...+........))',
 '........(.+.......)..',
 '........(.+........).',
 '........(.+.......)..',
 '........(.+........).',
 '........(.+.......)..',
 '........(.+........).',
 '........(.+.......)..',
 '........(.+........).',
 '..........+..........',
 '....(.((..+......)).)',
 '.....((...+........))',
 '........(.+.......)..',
 '........(.+........).',
 '........(.+.......)..',
 '........(.+........).',
 '..........+..........',
 '........(.+........).',
 '........(.+.......)..',
 '........(.+........).',
 '..........+..........',
 '........(.+.......)..',
 '........(.